# Introduction: #1 Data Query

This notebook shows how we download query results for `ga4_obfuscated_sample_ecommerce dataset` as a Pandas DataFrame, and export data in CSV file fomat `ecommerce_all.csv`.


## Dataset
[The data](https://developers.google.com/analytics/bigquery/web-ecommerce-demo-dataset) provides an obfuscated Google Analytics 4-property dataset for BigQuery, which contains business/eCommerce data from [Google Merchandise Store](https://www.googlemerchandisestore.com/), an online store that sells Google-branded merchandise.

The dataset includes the following kinds of information:
* __Traffic source data__: information about where website visitors originate. This includes data about organic traffic, paid search traffic, display traffic, etc.
* __Content data__: information about the behavior of users on the site. This includes the URLs of pages that visitors look at, how they interact with content, etc.
* __Transactional data__: information about the transactions that occur on the Google Merchandise Store website.


## Google Cloud Documentation
Before downloading the data, we have to prepare something in advance.

* Create and Manage service accounts and service account keys [Reference](https://cloud.google.com/iam/docs/creating-managing-service-accounts)
* Create credentials with scopes [Reference](https://cloud.google.com/bigquery/docs/samples/bigquery-auth-drive-scope)
* Download query results to DataFrame [Reference](https://cloud.google.com/bigquery/docs/samples/bigquery-query-results-dataframe)

## Imports
We are using a typical data science stack such as `pandas`, `numpy` and google cloud package like `bigquery`.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from datetime import date, timedelta

import os
from google.cloud import bigquery
from google.oauth2.service_account import Credentials

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 50)

## Download query results as a DataFrame
* BigQuery sample dataset for Google Analytics 4 ecommerce web implementation [Reference](https://developers.google.com/analytics/bigquery/web-ecommerce-demo-dataset)
* Introduction to Google Analytics 4 and advanced topics [Reference](https://developers.google.com/analytics/devguides/collection/ga4/ecommerce?client_type=gtag)
* Google Analytics 4 Events [Reference](https://developers.google.com/analytics/devguides/collection/ga4/reference/events)

In [2]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = \
'//Users//aprilchiu2000//Desktop//LINE//service_account_key//graphical-tape-338802-82aed3845959.json'

scopes = ('https://www.googleapis.com/auth/bigquery',
          'https://www.googleapis.com/auth/cloud-platform',
          'https://www.googleapis.com/auth/drive')

credentials = Credentials.from_service_account_file('/Users/aprilchiu2000/Desktop/LINE/service_account_key/graphical-tape-338802-82aed3845959.json')
credentials = credentials.with_scopes(scopes)
client = bigquery.Client(credentials=credentials)

In [3]:
query = \
"""
SELECT DISTINCT
    event_date,    
    FORMAT_TIMESTAMP('%Y-%m-%d %H:%M:%S', TIMESTAMP_MICROS(event_timestamp)) AS event_time,

    -- user demo/device
    user_pseudo_id,
    geo.continent,
    geo.sub_continent,
    geo.country,
    geo.region,
    geo.city,
    device.category AS device_category,
    device.mobile_brand_name,
    device.mobile_model_name,
    device.operating_system,
    device.is_limited_ad_tracking,
    device.web_info.browser AS web_browser,

    user_ltv.revenue,
    user_ltv.currency,

    -- traffic source
    traffic_source.name AS traffic_name,
    traffic_source.medium AS traffic_medium,
    traffic_source.source AS traffic_source,

    -- event
    (SELECT value.int_value FROM UNNEST(event_params) WHERE key = "ga_session_id") AS ga_session_id,
    event_name,
    event_value_in_usd,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = "page_title") AS page_title,
    (SELECT value.string_value FROM UNNEST(event_params) WHERE key = "page_location") AS page_location,
    ((SELECT value.int_value FROM UNNEST(event_params) WHERE key = "engagement_time_msec"))/1000 AS engagement_time_sec,

    -- ecommerce
    ecommerce.total_item_quantity,
    ecommerce.purchase_revenue_in_usd,
    ecommerce.refund_value_in_usd,
    ecommerce.unique_items,
    ecommerce.transaction_id,

    -- items
    items.item_id,
    items.item_name,
    items.item_brand,
    item_variant,
    items.item_category,
    items.item_category2,
    items.item_category3,
    items.item_category4,
    items.item_category5,
    items.price_in_usd,
    items.quantity,
    items.item_revenue_in_usd,
    items.item_refund_in_usd,
    items.coupon,
    items.affiliation,
    items.item_list_name,
    items.item_list_index,

    -- promotion
    items.promotion_id,
    items.promotion_name,
    items.creative_name,
    items.creative_slot

FROM `bigquery-public-data.ga4_obfuscated_sample_ecommerce.events_*`, UNNEST (items) AS items
"""

In [4]:
client = bigquery.Client() 
query_job = client.query(query)
df = query_job.to_dataframe()
print("query success:", df.shape[0], "rows")

query success: 3980420 rows


In [5]:
df.head()

,event_date,event_time,user_pseudo_id,continent,sub_continent,country,region,city,device_category,mobile_brand_name,...,item_revenue_in_usd,item_refund_in_usd,coupon,affiliation,item_list_name,item_list_index,promotion_id,promotion_name,creative_name,creative_slot
0,20201103,2020-11-03 11:04:19,8825068.2274309139,Europe,Southern Europe,(not set),(not set),(not set),desktop,Apple,...,NaN,NaN,(not set),(not set),Not available in demo dataset,15,(not set),(not set),(not set),(not set)
1,20201103,2020-11-03 07:43:31,10673312.8932527724,Americas,Northern America,United States,Georgia,(not set),desktop,Google,...,NaN,NaN,(not set),(not set),Not available in demo dataset,4,(not set),(not set),(not set),(not set)
2,20201103,2020-11-03 13:50:36,15626554.5768081465,Americas,Northern America,United States,New York,New York,mobile,Apple,...,NaN,NaN,(not set),(not set),Not available in demo dataset,5,(not set),(not set),(not set),(not set)
3,20201103,2020-11-03 22:34:53,16462431.2283594309,Americas,South America,Colombia,(not set),(not set),mobile,Apple,...,NaN,NaN,(not set),(not set),Not available in demo dataset,12,(not set),(not set),(not set),(not set)
4,20201103,2020-11-03 21:31:24,16765047.7521791154,Americas,South America,Ecuador,Pichincha,Quito,desktop,Google,...,NaN,NaN,(not set),(not set),Not available in demo dataset,9,(not set),(not set),(not set),(not set)


In [6]:
# df.to_csv('ecommerce_original.csv', index=False)

In [7]:
df.drop(columns=['is_limited_ad_tracking','currency','refund_value_in_usd',
                 'item_category2','item_category3','item_category4','item_category5','item_refund_in_usd',
                 'coupon','affiliation','item_list_name','item_list_index','promotion_id','creative_name','creative_slot']).to_csv('ecommerce_all.csv', index=False)